<a href="https://colab.research.google.com/github/axel-sirota/implement-nlp-word-embedding/blob/main/module4/Module4_Demo1_Finetuning_Glove_and_FastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi

Thu May 26 20:15:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Analysing Sentiment

Let's first import everything and load the dataset

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob, Word
import nltk
import torch
from torch import nn
import seaborn as sns
nltk.download('punkt')

%matplotlib inline
sns.set(rc={'figure.figsize':(20,20)})
import warnings
warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
%%writefile get_data.sh
if [ ! -f yelp.csv ]; then
  wget -O yelp.csv https://raw.githubusercontent.com/axel-sirota/implement-nlp-word-embedding/main/module3/data/yelp.csv
fi
if [ ! -f glove.6B.zip ]; then
  wget -O glove.6B.zip https://nlp.stanford.edu/data/glove.6B.zip
  unzip glove.6B.zip
fi
if [ ! -f cc.en.300.bin ]; then
  wget -O cc.en.300.bin.gz https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
  gzip -d cc.en.300.bin.gz
  cp cc.en.300.bin /usr/local/lib/python3.7/dist-packages/gensim/test/test_data
fi
ls

Overwriting get_data.sh


In [5]:
!bash get_data.sh


cc.en.300.bin		 glove.6B.100d.txt  glove.6B.50d.txt  yelp.csv
emb_word2vec_format.txt  glove.6B.200d.txt  glove.6B.zip
get_data.sh		 glove.6B.300d.txt  sample_data


In [6]:
CORPUS_SIZE = 100000
EMBEDDING_DIM = 300
path = 'yelp.csv'
yelp = pd.read_csv(path)
# Create a new DataFrame that only contains the 5-star and 1-star reviews.
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)].reset_index(drop=True)[:CORPUS_SIZE]
yelp_best_worst


,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
3,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0
4,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,7,7,4
...,...,...,...,...,...,...,...,...,...,...
495,ScvhAJe5lIJ7_SIvNTDfgg,2012-08-24,OT9779o629UEC8gmI8Fa4g,5,So yummy!! My favorite Mexican place!! Love th...,review,I7zvmDRYtsLUHsLi50wDNA,1,1,0
496,YSesarsmfOIKwx6eiRqFZQ,2009-04-30,x6b0jru4OlUzfzeoNxvZEg,5,I have been taking vehicles to this place for ...,review,1f8_7NDng8w5CV7bshe63A,0,1,0
497,5nahZe5bBYUbFWgEfwoNOA,2011-03-28,n-MKaPksVzxBoTlda0gYzA,5,Thank you Sunflower for having a decent select...,review,BfetZm9fa0zqyAFn8vo_6w,0,0,0
498,gUt-pPUpOVVhaCFC8-E4yQ,2009-01-21,fZ6ktQYONjrOcK4SjTxS8w,5,Private Karaoke at Geisha A Go Go is the coole...,review,8gM0VBG-5vlS7teGszclKQ,2,2,1


## Load GloVe weights and model

In [7]:
torch.cuda.empty_cache()

In [8]:
# convert glove to word2vec format
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file="glove.6B.300d.txt", word2vec_output_file="emb_word2vec_format.txt")

(400000, 300)

In [9]:
import gensim
import torch
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('emb_word2vec_format.txt')

In [10]:
weights_glove = torch.FloatTensor(word2vec_model.vectors).to(device)
tokenizer = lambda x: TextBlob(x).words

## Load FastText vectors

In [11]:
from gensim.models import fasttext
from gensim.test.utils import datapath

fasttext_model = fasttext.load_facebook_vectors(datapath("cc.en.300.bin"))

In [12]:
weights_fasttext= torch.FloatTensor(fasttext_model.vectors).to(device)

In [13]:
weights_glove.shape, weights_fasttext.shape

(torch.Size([400000, 300]), torch.Size([2000000, 300]))

In [14]:
vocab_size_glove = len(word2vec_model.index_to_key)
vocab_size_fasttext = len(fasttext_model.index_to_key)

In [15]:
vocab_size_glove, vocab_size_fasttext

(400000, 2000000)

In [16]:
def get_maximum_review_length(df):
  maximum = 0
  for ix, row in df.iterrows():
    candidate = len(tokenizer(row.text))
    if candidate > maximum:
      maximum = candidate
  return maximum

In [17]:
def get_train_test_split_by_embedding(embedding, df):
  maximum = get_maximum_review_length(df)
  X_pre_split = torch.zeros(len(df), maximum).type(torch.LongTensor).to(device)
  for index, row in df.iterrows():
    ix = 0
    for word in tokenizer(row.text):
      if word not in embedding:
        representation = 0
      else:
        representation = embedding.index_to_key.index(word)
      X_pre_split[index, ix] = representation
      ix += 1
  X = X_pre_split
  y = yelp_best_worst.stars.map({1:0, 5:1})
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
  y_train = torch.Tensor(y_train.values).type(torch.LongTensor).to(device)
  y_test = torch.Tensor(y_test.values).type(torch.LongTensor).to(device)
  return X_train, X_test, y_train, y_test

In [18]:
X_train, X_test, y_train, y_test = get_train_test_split_by_embedding(word2vec_model, yelp_best_worst)

In [19]:
class SentimentPretrained(nn.Module):
    def __init__(self, embedding_dim, weights):
        super(SentimentPretrained, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding.from_pretrained(weights, freeze=False)
        self.linear = nn.Linear(embedding_dim, 2)
        self.activation_function = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = torch.sum(self.embeddings(inputs), dim=1).squeeze()
        out = self.linear(embeds.float())
        out = self.activation_function(out)
        return out

In [20]:
def loss(y_pred, y):
  return nn.functional.nll_loss(y_pred, y)

def metric(y_pred, y):  # -> accuracy
  return (1 / len(y)) * ((y_pred.argmax(dim = 1) == y).sum())

## Let's verify the metric makes sense

In [21]:
X_train[5]

tensor([    0,  1409,  1290,  2821,   285,   197,   219,    44, 22078,    32,
            0,   977,    20,    34,    20,   119,    70,    88,  1096,    10,
          285,     0, 12611, 20890,   117,   645,    81,    86,   169,    22,
            0,    34,  1495,     0,   626,     0,   134,   175,     7,     0,
            3,     0,     0,     0,   287,     0,   565,     4,  4374,   117,
           36,   120,   130,  4018, 12611,     0,     0,    40,     0,  5795,
           17, 60128, 15235,    13,     7, 71585,    17,   232,  1051,     3,
        12617,    48,    15, 30580,     5,     0,    68,    15, 38877,     0,
           20,    15,     0, 71585,  5872,     0,   269,     0, 30580, 12617,
           15,   116,    63,    34,     0,   346,    10,  1085,     0, 38877,
           15,    36,   116,    63,    20, 20890,   117,   645,     0,    40,
           22,     7, 13710,    42,    15,     6,     7, 36472,     0,    15,
           70,  1096,     4, 36887,   163,    59,     0,   238, 

In [22]:
weights_glove.shape

torch.Size([400000, 300])

In [23]:
nn.Embedding.from_pretrained(weights_glove)(X_train)

tensor([[[ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
         [ 0.1825, -0.0466, -0.3471,  ...,  0.2344,  0.3991, -0.0146],
         [ 0.0385, -0.0398,  0.0827,  ..., -0.3343,  0.0118,  0.0597],
         ...,
         [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
         [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
         [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539]],

        [[ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
         [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
         [-0.2044,  0.1643,  0.0418,  ..., -0.3401, -0.0771, -0.0841],
         ...,
         [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
         [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
         [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539]],

        [[ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
         [-0.1067, -0.2527, -0.2965,  ..., -0

In [24]:
y_train_pred = SentimentPretrained(300, weights_glove).to(device)(X_train).to(device)
y_train_pred.argmax(dim = 1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [25]:
y_train

tensor([1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
        1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,

In [26]:
(y_train_pred.argmax(dim = 1) == y_train).sum()

tensor(87, device='cuda:0')

In [27]:
metric(y_train_pred, y_train[:CORPUS_SIZE])

tensor(0.2175, device='cuda:0')

In [28]:
del y_train_pred

## The training routine, training all 4 embeddings

In [29]:
def train_model_with_embedding(weights, X_train, X_test, y_train, y_test, embedding_dimension=EMBEDDING_DIM):
  model = SentimentPretrained(embedding_dimension, weights).to(device)
  optimizer = torch.optim.AdamW(model.parameters())
  epochs = 250
  for i in range(epochs):
    y_pred = model(X_train)
    xe = loss(y_pred, y_train)
    accuracy = metric(y_pred, y_train)
    xe.backward()
    if i % 50 == 0:
      print("Loss: ", xe, " Accuracy ", accuracy.data.item())
    optimizer.step()
    optimizer.zero_grad()
  y_test_pred = model(X_test)
  print(f'Model accuracy is {metric(y_test_pred, y_test)}')

In [30]:
train_model_with_embedding(weights_glove, X_train, X_test, y_train, y_test)

Loss:  tensor(22.3060, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7824999690055847
Loss:  tensor(0.1174, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.9449999928474426
Loss:  tensor(0.0401, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.9899999499320984
Loss:  tensor(0.0282, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.9950000047683716
Loss:  tensor(0.0215, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.9950000047683716
Model accuracy is 0.9300000071525574


In [31]:
del weights_glove
torch.cuda.empty_cache()

In [32]:
train_model_with_embedding(weights_fasttext, X_train, X_test, y_train, y_test)

Loss:  tensor(51.8280, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.2175000011920929
Loss:  tensor(0.0691, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.9724999666213989
Loss:  tensor(0.0351, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.9925000071525574
Loss:  tensor(0.0249, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.9950000047683716
Loss:  tensor(0.0189, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.9950000047683716
Model accuracy is 0.8700000047683716


In [33]:
del weights_fasttext
torch.cuda.empty_cache()

Now we will generate the train and test sets for the OHE and TFIdf

In [38]:
def get_tensor_reps_for_traditional_embeddings(klass):
  X = yelp_best_worst.text
  y = yelp_best_worst.stars.map({1:0, 5:1})
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
  vect = klass(min_df=5, stop_words='english', strip_accents='ascii', analyzer='word', max_features=300)  # in order to compare similarly
  X_train_dtm = vect.fit_transform(X_train)
  X_test_dtm = vect.transform(X_test)
  X_train_tensor = torch.Tensor(X_train_dtm.toarray()).to(device)
  X_test_tensor = torch.Tensor(X_test_dtm.toarray()).to(device)
  y_train = torch.Tensor(y_train.values).type(torch.LongTensor).to(device)
  y_test = torch.Tensor(y_test.values).type(torch.LongTensor).to(device)
  return vect, X_train_tensor, X_test_tensor, y_train, y_test

In [39]:
vect_ohe, X_train_tensor_ohe, X_test_tensor_ohe, y_train, y_test = get_tensor_reps_for_traditional_embeddings(CountVectorizer)
X_train_tensor_ohe = X_train_tensor_ohe.type(torch.LongTensor).to(device)
X_test_tensor_ohe = X_test_tensor_ohe.type(torch.LongTensor).to(device)
weights_ohe = torch.from_numpy(np.eye(len(vect_ohe.get_feature_names())))
X_train_tensor_ohe.shape, X_test_tensor_ohe.shape, weights_ohe.shape

(torch.Size([400, 300]), torch.Size([100, 300]), torch.Size([300, 300]))

In [40]:
train_model_with_embedding(weights_ohe, X_train_tensor_ohe, X_test_tensor_ohe, y_train, y_test)

Loss:  tensor(0.6207, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7874999642372131
Loss:  tensor(0.5056, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7849999666213989
Loss:  tensor(0.5002, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7899999618530273
Loss:  tensor(0.4984, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7899999618530273
Loss:  tensor(0.4977, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7899999618530273
Model accuracy is 0.8299999833106995


In [41]:
vect_tfidf, X_train_tensor_tfidf, X_test_tensor_tfidf, y_train, y_test = get_tensor_reps_for_traditional_embeddings(TfidfVectorizer)
X_train_tensor_tfidf = X_train_tensor_tfidf.type(torch.LongTensor).to(device)
X_test_tensor_tfidf = X_test_tensor_tfidf.type(torch.LongTensor).to(device)
weights_tfidf = torch.from_numpy(np.eye(len(vect_tfidf.get_feature_names())))

In [42]:
train_model_with_embedding(weights_tfidf, X_train_tensor_tfidf, X_test_tensor_tfidf, y_train, y_test)

Loss:  tensor(1.8317, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7824999690055847
Loss:  tensor(0.5255, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7824999690055847
Loss:  tensor(0.5236, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7824999690055847
Loss:  tensor(0.5236, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7824999690055847
Loss:  tensor(0.5236, device='cuda:0', grad_fn=<NllLossBackward0>)  Accuracy  0.7824999690055847
Model accuracy is 0.8700000047683716
